In [10]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

# **Searching in 'Power Law' Network**

In [ ]:
 !wget https://snap.stanford.edu/data/p2p-Gnutella31.txt.gz
 !gunzip p2p-Gnutella31.txt.gz
 

In [ ]:
!wget https://snap.stanford.edu/data/facebook_combined.txt.gz
!gunzip facebook_combined.txt.gz

In [13]:
#function to get the largest connected subgraph of diffrent node size
def get_subgraph(G,node_list,nodes):
  G1=G.subgraph(node_list[:nodes])
  connected_component=[G1.subgraph(c) for c in nx.connected_components(G1)]
  G1=max(connected_component,key= lambda x:x.number_of_nodes())
  return G1

# applying searching algorithm
# random=True means applying random search
# random = False mean applying seaching utilizing hiegher degree nodes
def search(G,visited,start,target,random=False):
  current_node,step=start,0
  parent_list=[start]
  visited[start]=True
  
  while True:
    neighbours=[]
    for node in G.neighbors(current_node):
      # if target found retrurn the step taken
      if node==target: 
        return step
      elif node!=parent_list[-1] and not visited[node]:
        neighbours.append(node)
      else: pass
  
    # next node selction based on random walk or utilizing higher degree node
    if len(neighbours)>0:
      next_node=0
      # random selection
      if random:
        next_node=np.random.choice(neighbours)
      else:
        degree=G.degree(neighbours)
        # select next node with maximum degree
        next_node=max(degree, key=lambda x:x[1])[0] 
      
      visited[next_node]=True
      parent_list.append(current_node)
      current_node=next_node 
      step+=1;

    else:
      step+=1
      # if all child are visited and node not found go to parant node and selcet onother next node based on searching streatgy
      current_node=parent_list[-1]
      parent_list.pop()

# function to find the average search time 
def average_search_time(G,search_list,n,random=False):

  total_step,node_explored =0,0
  # running searching algorithm for different start and target node
  for start,target in search_list:
    visited=[False]*n
    total_step+=search(G,visited,start,target,random)
    node_explored+=sum(visited)
  # taking average of all search time
  average=total_step//len(search_list)
  average_explored=node_explored//len(search_list)
  
  return average_explored,average


In [14]:

def run(G,node_list,smallest,step,largest):
  step_random=[]
  step_dseq=[]
  no_nodes=[]
  frac_explored_r,frac_explored_d=[],[]
  for idx,nodes in enumerate(list(range(smallest,largest,step))):
    G1=get_subgraph(G,node_list,nodes)
    search_list=list(G1.nodes)
    np.random.shuffle(search_list)
    n1=G1.number_of_nodes()
    search_count=(n1//(4*(idx+1)))*2
    ##print("Search Count ",search_count,len(no_nodes))

    search_list=np.array(search_list[:search_count]).reshape(-1,2)
    ave_node_r,step_r=average_search_time(G1,search_list,largest,True)
    step_random.append(step_r)
    frac_explored_r.append(ave_node_r/n1)
    ##print("Using Random Search",step_r,ave_node_r)
    ave_node_d,step_d=average_search_time(G1,search_list,largest)
    step_dseq.append(step_d)
    frac_explored_d.append(ave_node_d/n1)
    ##print("Using Degree Sequence",step_d,ave_node_d)
    no_nodes.append(n1)


  return step_random,step_dseq,frac_explored_r,frac_explored_d,no_nodes


In [15]:

def read_write(filename,final_list=None,write=False):
  if write:
    string=""
    for i in final_list:
      str1=' '.join("%s"%e for e in i)
      string+='\n'+str1
    with open('/content/drive/My Drive/sna/'+filename,'w') as file:
      file.write(string)
  else:
    file=open('/content/'+filename).readlines()
    lines=[i.strip("\n") for i in file if len(i.strip("\n"))>0]

    random_search=[int(i) for i in lines[0].split(" ")]
    degree_sequence=[int(i) for i in lines[1].split(" ")]
    node_exp_r=[round(float(i),2) for i in lines[2].split(" ")]
    node_exp_d=[round(float(i),2) for i in lines[3].split(" ")]
    no_nodes=[int(i) for i in lines[4].split(" ")]
    plot(random_search,degree_sequence,node_exp_r,node_exp_d,no_nodes)

  



In [16]:

def plot(random_search,degree_sequence,node_exp_r,node_exp_d,no_nodes):
  plt.plot(no_nodes,random_search,c='r',label='Using Random Search')
  plt.plot(no_nodes,degree_sequence,c='g',label='Using Degree Sequence')
  plt.title("Comparison of Average Search Time")
  plt.xlabel("Size of Graph")
  plt.ylabel("Average Search Time")
  plt.legend()
  plt.show()
  plt.plot(no_nodes,node_exp_r,c='r',label='Using Random Search')
  plt.plot(no_nodes,node_exp_d,c='g',label='Using Degree Sequence')
  plt.title("Comparison of fraction of node explored")
  plt.xlabel("Size of Graph")
  plt.ylabel("Fraction of Node explored")
  plt.legend()
  plt.show()


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Gnutella network
G=nx.read_edgelist('/content/p2p-Gnutella31.txt',nodetype=int)
degree=nx.degree(G)
node_sorted=sorted(degree,key=lambda x:x[1],reverse=True)
node_list=[i[0] for i in node_sorted]
n=G.number_of_nodes()
print("graph size = ",n)
step_random,step_dseq,frac_explored_r,frac_explored_d,no_nodes=run(G,node_list,1000,4000,n)
final_list=[step_random,step_dseq,frac_explored_r,frac_explored_d,no_nodes]

read_write('plot (1).txt')


graph size =  62586


In [ ]:
# facebook network
G=nx.read_edgelist('/content/facebook_combined.txt',nodetype=int)
degree=nx.degree(G)
node_sorted=sorted(degree,key=lambda x:x[1],reverse=True)
node_list=[i[0] for i in node_sorted]
n=G.number_of_nodes()
print("graph size = ",n)

step_random,step_dseq,frac_explored_r,frac_explored_d,no_nodes=run(G,node_list,1000,500,n)
final_list=[step_random,step_dseq,frac_explored_r,frac_explored_d,no_nodes]
read_write('plot (2).txt',final_list,True)
